<a href="https://colab.research.google.com/github/wilzamguerrero/SDZ/blob/main/SDZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Buy Me a cofee](https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/coffe4.gif?raw=true ) ](https://www.buymeacoffee.com/wilzamguerrero)

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/drive.png?raw=true' height="50" alt="netdata"/></center>
from IPython.display import clear_output
from google.colab import drive
drive.mount('/content/drive')
clear_output()
print('[1;32mDrive Connected ✓')



#markdown Download tool
#import os
#from IPython.display import clear_output

#save_models_dir = "/content/sample_data"#@param {type:"string"}
#os.chdir(save_models_dir)
#url = "https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt" #@param {type:"string"}
#!wget --no-check-certificate $url

#clear_output()
#print('[1;32mModel Save ✓')
     

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZanim2.gif?raw=true' height="50" alt="netdata"/></center>
##@markdown <left><font face='Squired Regular' color='#00f3f3' size='4'>Stable Diffusion Z</font>

import os
import shutil
import urllib.request
#from IPython.utils import capture
from IPython.utils.capture import capture_output
from IPython.display import clear_output
import sys
import fileinput
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from ipywidgets import interact, interactive, fixed
from ipywidgets import Layout, SelectMultiple
from IPython.core.display import HTML

SDZ_install = "/content/sample_data"
SDZ_install_widget = widgets.Text(value=SDZ_install, description="SDZ_install")
SDZ_Version = widgets.Dropdown(options=["V1.5", "V2.1"], description="Version")
Download_Base_Model = widgets.Dropdown(options=["", "V1.5", "V1.5.inpaint", "V1.5.pix2pix", "V2.1"], description="Model")
SDZ = "stable-diffusion-webui"
SDZ_widget = widgets.Text(value=SDZ, description="SDZ")
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
SDZ_webui_path_widget = widgets.Text(value=SDZ_webui_path, description="SDZ_webui_path")
model_init = widgets.Text(value="", description="Model_init")
models_dir = widgets.Text(value="", description="Models_dir")
SDZ_Args = widgets.Text(value="", description="Args")
SDZ_Training = widgets.Dropdown(options=["", "Kohya", "New_Dreambooth", "Old_Dreambooth"], description="Training")
SDZ_ControlNet = widgets.Checkbox(value=False, description="ControlNet")
SDZ_Extensions = widgets.Checkbox(value=False, description="Extensions")
SDZ_Upscaler = widgets.Checkbox(value=False, description="Upscalers")
SDZ_Big_Models = widgets.Checkbox(value=False, description="Big_Models")
SDZ_Continue = widgets.Checkbox(value=False, description="🔵 SDZ CONTINUE")
SDZconfigV2="--config v2-inference-v.yaml"

os.chdir(SDZ_install_widget.value)
boton_runSDZ = widgets.Button(description="⭕ RUN > > > SDZ")
boton_runSDZ.style.button_color = '#1e1e1e'
boton_runSDZ.style.font_weight = 'bold'
boton_runSDZ.style.color = 'black'

widgets_box = widgets.VBox([SDZ_install_widget, SDZ_Version, Download_Base_Model,],layout=widgets.Layout(height='120px', width='100%'))
final_box = widgets.VBox([model_init, models_dir, SDZ_Args, SDZ_Training],layout=widgets.Layout(height='160px', width='100%'))
final_box1 = widgets.VBox([SDZ_Extensions, SDZ_ControlNet, SDZ_Upscaler, SDZ_Big_Models],layout=widgets.Layout(height='170px', width='100%'))
final_box2 = widgets.VBox([SDZ_Continue],layout=widgets.Layout(height='90px', width='100%'))

# SDZ_RunSDZ
def runSDZ(b):
    global widgets_box, boton_runSDZ
    widgets_box.close()
    final_box.close()
    final_box1.close()
    final_box2.close()
    boton_runSDZ.close()
    share = "--share --api --cors-allow-origins=https://www.painthua.com --xformers --theme=dark --disable-safe-unpickle --enable-insecure-extension-access --no-half-vae" + " --ckpt=" + model_init.value + " --ckpt-dir=" + models_dir.value + " " + SDZ_Args.value

    if not SDZ_Continue.value:
      with capture_output() as cap:
        import os
        if os.path.exists(SDZ_widget.value):
          shutil.rmtree(SDZ_widget.value)
        !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

        import os
        if not os.path.exists(SDZ_widget.value):
          !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

        %cd stable-diffusion-webui
        !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/Upscalers.txt
        !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
        !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css

        clear_output()
      print('[1;96mInstalling SDZ ✓')

    if SDZ_Continue.value:
      with capture_output() as cap:
        import os
        os.chdir(SDZ_webui_path_widget.value)
        clear_output()
      print('[1;96mPreparing SDZ Continue ✓')

    with capture_output() as cap:
      import os
      os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
      !pip install pytorch_lightning==1.7.7
      !pip install triton==2.0.0.dev20221202.
      !wget -N https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
      !git clone https://github.com/bytedance/Next-ViT.git externals/Next_ViT

# SDZ_Models 
      os.chdir(SDZ_webui_path_widget.value)
      %cd models/Stable-diffusion
      selected_option = Download_Base_Model.value
      if selected_option == "V1.5":
        !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

      elif selected_option == "V1.5.inpaint":
        !wget https://huggingface.co/uwg/modelz_base/resolve/main/v1-5-inpaint-fp16.ckpt -O v15inp.ckpt

      elif selected_option == "V1.5.pix2pix":
        !wget https://huggingface.co/timbrooks/instruct-pix2pix/resolve/main/instruct-pix2pix-00-22000.safetensors -O v15pix2pix.safetensors

      elif selected_option == "V2.1":
        !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt

      else: selected_option == ""
      os.chdir(SDZ_webui_path_widget.value)

# SDZ_Extensions
      if SDZ_Extensions.value:
        os.chdir(SDZ_webui_path_widget.value)
        %cd extensions
        !git clone https://github.com/Extraltodeus/depthmap2mask.git
        !git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111.git
        !git clone https://github.com/camenduru/sd-civitai-browser.git
        !git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git
        !git clone https://github.com/fkunn1326/openpose-editor.git
        !git clone https://github.com/Gerschel/sd-web-ui-quickcss.git
        !git clone https://github.com/zero01101/openOutpaint-webUI-extension.git
        !sed -i 's/dl_url = gr.Textbox(label="Download Url", interactive=False, value=None)/dl_url = gr.Textbox(label="Download Url", value=None, interactive=True, lines=1)/' $SDZ_install'/stable-diffusion-webui/extensions/sd-civitai-browser/scripts/civitai-api.py'
        %cd ..

# SDZ_ControlNet
      if SDZ_ControlNet.value:
        os.chdir(SDZ_webui_path_widget.value)
        %cd extensions
        !git clone https://github.com/Mikubill/sd-webui-controlnet.git
        %cd sd-webui-controlnet/models
        !git clone https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors
        !git clone https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors
        !git clone https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors
        !git clone https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors
        !git clone https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors
        !git clone https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors
        !git clone https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors
        !git clone https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors
        !git clone https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_keypose-fp16.safetensors
        !git clone https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_seg-fp16.safetensors
        !git clone https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_sketch-fp16.safetensors
        %cd ../../..

# SDZ_Upscalers
      if SDZ_Upscaler.value:
        os.chdir(SDZ_webui_path_widget.value+'/'+'models')

        if not os.path.exists('ESRGAN'):
            os.makedirs('ESRGAN')
        with open(SDZ_webui_path_widget.value+'/'+'Upscalers.txt') as f:
            for i, line in enumerate(f):
                if 1 <= i <= 34:
                    link = line.strip()
                    os.system('wget --timestamping -P ESRGAN {}'.format(link))

        if not os.path.exists('GFPGAN'):
            os.makedirs('GFPGAN')
        with open(SDZ_webui_path_widget.value+'/'+'Upscalers.txt') as f:
            for i, line in enumerate(f):
                if 36 <= i <= 38:
                    link = line.strip()
                    os.system('wget --timestamping -P GFPGAN {}'.format(link))

        if not os.path.exists('SwinIR'):
            os.makedirs('SwinIR')
        with open(SDZ_webui_path_widget.value+'/'+'Upscalers.txt') as f:
            for i, line in enumerate(f):
                if 40 <= i <= 54:
                    link = line.strip()
                    os.system('wget --timestamping -P SwinIR {}'.format(link))

# SDZ_Training
      selected_option = SDZ_Training.value
      if selected_option == "Kohya":
        os.chdir(SDZ_webui_path_widget.value)
        %cd extensions
        !git clone https://github.com/ddpn08/kohya-sd-scripts-webui.git
        %cd ..

      elif selected_option == "New_Dreambooth":
        os.chdir(SDZ_webui_path_widget.value)
        %cd extensions
        !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
        %cd ..

      elif selected_option == "Old_Dreambooth":
        os.chdir(SDZ_webui_path_widget.value)
        if not os.path.exists('/usr/local/lib/python3.10/site-packages/open_clip'):
          !wget -N https://github.com/korakot/kora/releases/download/v0.10/py310.sh
          !bash ./py310.sh -b -f -p /usr/local
          !python -m ipykernel install --name "py310" --user
          !conda install -c "xformers/label/dev" xformers -y
          !pip install open-clip-torch
          !pip install triton
          !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
        os.chdir(SDZ_install_widget.value)
        %cd stable-diffusion-webui
        !git checkout 1130d5df669911a5c67696be90bccca3ecf5f487
        !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
        !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user_old.css -O user.css
        %cd extensions
        !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
        %cd sd_dreambooth_extension
        !git checkout f9900ccb3ce7025d1c89334e320a1c3b255bbd07
        %cd ../..

      else: selected_option == ""
      os.chdir(SDZ_webui_path_widget.value)

# SDZ_Big_Models
      if SDZ_Big_Models.value:
        !sed -i 's@cmd_opts.lowram else \"cpu\"@cmd_opts.lowram else \"cuda\"@' $SDZ_webui_path'/modules/shared.py'
        !sed -i "s@map_location='cpu'@map_location='cuda'@" $SDZ_webui_path'/modules/extras.py'
        
        !sed -i '902s@.*@        self.logvar = self.logvar.to(self.device)@' $SDZ_webui_path'/repositories/stable-diffusion-stability-ai/ldm/models/diffusion/ddpm.py'
        !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' $SDZ_webui_path'/webui.py'
        !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda' if os.path.getsize(checkpoint_file) > 5500000000 else map_location@" $SDZ_webui_path'/modules/sd_models.py'
      else:
        !sed -i 's@cmd_opts.lowram else \"cuda\"@cmd_opts.lowram else \"cpu\"@' $SDZ_webui_path'/modules/shared.py'
        !sed -i "s@map_location='cuda'@map_location='cpu'@" $SDZ_webui_path'/modules/extras.py'

        clear_output()
    print('[1;90mPreparing SDZ System ✓')

    selected_option = SDZ_Version.value
    if selected_option == "V1.5":
        !python launch.py $share
    elif selected_option == "V2.1":
        !python launch.py $share $SDZconfigV2

boton_runSDZ.on_click(runSDZ)
group_title2 = widgets.HTML(value='')
display(widgets_box, group_title2, final_box, final_box1, final_box2, boton_runSDZ)